In [1]:
import torch
from torch import nn
from entmax import entmax_bisect
from torch.autograd import grad
from tqdm import tqdm
#from entmax.activations import sparsemax, entmax15

In [2]:
#alpha = nn.Parameter(torch.Tensor([1.5]))
#alpha

In [3]:
class AlphaChooser(torch.nn.Module):

    def __init__(self, head_count):
        """head_count (int): number of attention heads"""
        super(AlphaChooser, self).__init__()
        self.pre_alpha = nn.Parameter(torch.randn(head_count))

    def forward(self):
        alpha = 1 + torch.sigmoid(self.pre_alpha)
        return torch.clamp(alpha, min=1.01, max=2)

In [155]:
class EntmaxAlpha(nn.Module):

    def __init__(self, head_count, dim=0):
        super(EntmaxAlpha, self).__init__()
        self.dim = dim
        self.alpha_chooser = nn.Parameter(AlphaChooser(head_count)())

    def forward(self, att_scores):
        batch_size, head_count, query_len, key_len = att_scores.size()

        self.alpha = self.alpha_chooser
        expanded_alpha = self.alpha.unsqueeze(0).unsqueeze(-1).unsqueeze(-1) # [1,nb_heads,1,1]
        expanded_alpha = expanded_alpha.expand((batch_size, -1, query_len,1))# [bs, nb_heads, query_len,1]
        p_star = entmax_bisect(att_scores, expanded_alpha)

        return p_star

In [156]:
entmax_alpha = EntmaxAlpha(12)

In [157]:
att_scores = torch.rand(128, 12, 36, 20)

In [158]:
entmax_alpha(att_scores).shape

torch.Size([128, 12, 36, 20])

In [ ]:
alpha_chooser = AlphaChooser(12)

In [114]:
bs, head_count, query_len, key_len = att_scores.size()

In [115]:
#att_scores.view(-1, key_len).shape

In [116]:
#att_scores = att_scores.view(-1, key_len)

In [117]:
alpha_chooser().unsqueeze(0).unsqueeze(-1).unsqueeze(-1).shape

torch.Size([1, 12, 1, 1])

In [124]:
expanded_alpha = alpha_chooser().unsqueeze(0).unsqueeze(-1).unsqueeze(-1)

In [126]:
expanded_alpha.shape

torch.Size([1, 12, 1, 1])

In [127]:
expanded_alpha.expand((128,-1,query_len,1)).shape

torch.Size([128, 12, 36, 1])

In [120]:
expanded_alpha = expanded_alpha.expand((128,-1,query_len,1))

In [121]:
#expanded_alpha.contiguous().view(-1).shape

In [122]:
att_scores.shape, expanded_alpha.shape

(torch.Size([128, 12, 36, 20]), torch.Size([128, 12, 36, 1]))

In [123]:
entmax_bisect(att_scores, expanded_alpha).shape

torch.Size([128, 12, 36, 20])